In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molsysmt as msm
import openexplorer as oe
import numpy as np
from simtk import unit
from simtk.openmm import app

In [3]:
modeller = msm.convert('alanine_tetrapeptide.pdb', to_form='openmm.Modeller')

topology = modeller.topology
positions = modeller.positions

forcefield = app.ForceField('amber10.xml', 'amber10_obc.xml')
system = forcefield.createSystem(topology, nonbondedMethod=app.NoCutoff, #constraints=app.HBonds,
                                implicitSolvent=app.OBC2, soluteDielectric=1.0, solventDielectric=78.5)

In [4]:
explorer = oe.Explorer(topology, system, platform='CUDA')
explorer.set_coordinates(positions)
explorer.md.langevin(1000)
explorer.quench.l_bfgs()

In [5]:
temperature = 500.0*unit.kelvin
timestep = 2.0*unit.femtoseconds
collision_rate = 1.0/unit.picoseconds # Openmm suggests 1 ps-1 but will it be the same for implicit solvent? 64?
explorer.md.langevin.set_parameters(temperature=temperature, timestep=timestep,
                                    collision_rate=collision_rate)

threshold = 0.01*unit.angstroms

time_langevin_cicle = 1.0*unit.picoseconds
steps_langevin_cicle = int(time_langevin_cicle/timestep)
n_attempts_per_minimum = 250

minima_coordinates = []
minima_potential_energy = []
n_minima = 0

explorer.quench.l_bfgs()
minima_coordinates.append(explorer.get_coordinates())
minima_potential_energy.append(explorer.get_potential_energy())
n_minima += 1


current_minimum_index = 0

while current_minimum_index < n_minima:
    
    current_minimum_coordinates = minima_coordinates[current_minimum_index]
    n_attempts = 0
    n_cicles = 0
    
    coordinates = current_minimum_coordinates
    explorer.set_coordinates(coordinates)
    explorer.set_velocities_to_temperature(temperature)
    
    while n_attempts<n_attempts_per_minimum:
    
        pre_coordinates=explorer.get_coordinates()
        
        explorer.md.langevin(steps_langevin_cicle)
        coordinates = explorer.get_coordinates()
        velocities = explorer.get_velocities()
        
        explorer.quench.l_bfgs()
    
        if explorer.distance.least_rmsd(current_minimum_coordinates) <= threshold:
            explorer.set_coordinates(coordinates)
            explorer.set_velocities(velocities)

        else:
        
            n_attempts+=1
        
            new_minimum=True
            for minimum_index in range(n_minima):
                if minimum_index!=current_minimum_index:
                    if explorer.distance.least_rmsd(minima_coordinates[minimum_index]) <= threshold:
                        new_minimum = False
                        break
            if new_minimum:
                minima_coordinates.append(explorer.get_coordinates())
                minima_potential_energy.append(explorer.get_potential_energy())
                n_minima+=1
        
            explorer.set_coordinates(pre_coordinates)
            explorer.set_velocities_to_temperature(temperature)

        n_cicles+=1
    
    print(current_minimum_index, n_cicles, n_minima)
    
    current_minimum_index+=1

0 420 8
1 1243 13
2 425 13
3 250 13
4 250 13
5 324 15
6 251 15
7 251 15
8 250 16
9 250 16
10 294 43
11 251 43
12 253 46
13 251 46
14 274 56
15 250 56
16 318 59
17 274 65
18 260 71
19 323 73
20 261 77
21 264 78
22 263 79
23 282 82
24 267 83
25 251 85
26 257 85
27 262 85
28 263 86
29 254 87
30 250 89
31 253 90
32 258 91
33 258 91
34 271 92
35 255 92
36 259 97
37 253 97
38 256 98
39 264 99
40 257 99
41 250 101
42 253 101
43 256 102
44 259 103
45 250 103
46 265 105
47 260 106
48 253 108
49 264 109
50 331 110
51 259 110
52 254 110
53 253 111
54 255 111
55 250 111
56 251 111
57 250 111
58 250 111
59 253 111
60 251 111
61 272 111
62 253 112
63 250 113
64 250 113
65 259 113
66 284 114
67 258 114
68 250 114
69 254 114
70 250 114
71 252 114
72 250 114
73 254 114
74 250 114
75 250 114
76 251 114
77 255 115
78 250 116
79 253 117
80 250 118
81 250 119
82 251 120
83 250 120
84 252 121
85 251 121
86 251 121
87 251 121
88 250 121
89 250 121
90 250 121
91 251 121
92 276 121
93 251 121
94 250 121
95 253

Podemos calcular la distancia entre configuraciones para hacer un mapa por proximidad: mds o red con threshold.

In [7]:
minima_potential_energy

[Quantity(value=-513.0481665288919, unit=kilojoule/mole),
 Quantity(value=-513.0533865581251, unit=kilojoule/mole),
 Quantity(value=-513.0474870612152, unit=kilojoule/mole),
 Quantity(value=-499.6362446513659, unit=kilojoule/mole),
 Quantity(value=-499.6312496465441, unit=kilojoule/mole),
 Quantity(value=-513.0363805488714, unit=kilojoule/mole),
 Quantity(value=-499.63561338719023, unit=kilojoule/mole),
 Quantity(value=-499.63387269435646, unit=kilojoule/mole),
 Quantity(value=-499.6180585909525, unit=kilojoule/mole),
 Quantity(value=-499.6349569397561, unit=kilojoule/mole),
 Quantity(value=-503.4596372664929, unit=kilojoule/mole),
 Quantity(value=-499.63174989438494, unit=kilojoule/mole),
 Quantity(value=-503.45418070812946, unit=kilojoule/mole),
 Quantity(value=-499.6290755472104, unit=kilojoule/mole),
 Quantity(value=-503.45978161111145, unit=kilojoule/mole),
 Quantity(value=-499.58956913864483, unit=kilojoule/mole),
 Quantity(value=-503.4669098738723, unit=kilojoule/mole),
 Quantit

In [8]:
0<inf

[autoreload of openexplorer.exploration_campaign.quench_and_restore failed: Traceback (most recent call last):
  File "/home/diego/Myopt/miniconda3/envs/UIBCDF_lab_dev/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/diego/Myopt/miniconda3/envs/UIBCDF_lab_dev/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/diego/Myopt/miniconda3/envs/UIBCDF_lab_dev/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/diego/Myopt/miniconda3/envs/UIBCDF_lab_dev/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_exter

NameError: name 'inf' is not defined

In [9]:
import numpy as np

In [10]:
1<np.inf

True